<a href="https://colab.research.google.com/github/jackschedel/AutoCalibr/blob/main/AutoCalibr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.5 MB/s eta 0:00:00


In [2]:
# mount Google Drive to VM disk
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
import os
import bpy

FBX = '/content/drive/MyDrive/datasets/AutoCalibr/fbx/'
PLY = '/content/drive/MyDrive/datasets/AutoCalibr/ply/'
DIR = '/content/drive/MyDrive/datasets/AutoCalibr/'

In [4]:
#@title { vertical-output: true}
!rm -r /content/drive/MyDrive/datasets/AutoCalibr/ply
!mkdir /content/drive/MyDrive/datasets/AutoCalibr/ply

for f in os.listdir(FBX):
    if f.endswith('.fbx'):
        # Delete all mesh objects to avoid exporting multiple models into the same file
        bpy.ops.object.select_all(action='DESELECT')
        bpy.ops.object.select_by_type(type='MESH')
        bpy.ops.object.delete()

        # Load in FBX file
        bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

        # Select the object
        obj_object = bpy.context.selected_objects[0]
        bpy.context.view_layer.objects.active = obj_object

        # Export object to PLY
        bpy.ops.export_mesh.ply(filepath=os.path.join(PLY, f.replace('.fbx', '.ply')), use_ascii=True, use_mesh_modifiers=True, use_normals=False, use_uv_coords=False, use_colors=False)

FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Coldheart.ply' in 0.822
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Vigilance Wing.ply' in 0.294
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/RatKing.ply' in 0.438
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Riskrunner.ply' in 0.513
FBX version: 7400
Export completed "/content/drive/MyDrive/datasets/AutoCalibr/ply/Skyburner's oath.ply" in 0.049
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Prometheus Lens.ply' in 0.148
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Tractor Cannon.ply' in 0.409
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Wardcliff Coil.ply' in 0.308
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Anarchy.ply' in 0.271
FBX version:

In [38]:
class Vertex:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Face:
    def __init__(self, vertices):
        self.vertices = vertices

class PlyObject:
    def __init__(self, name, vertices, faces):
        self.name = name
        self.vertices = vertices
        self.faces = faces


    def save_file(self, filename):
      with open(filename, "w") as file:
          file.write("ply\n")
          file.write("format ascii 1.0\n")
          file.write("comment Created by PlyObject class\n")
          file.write(f"element vertex {len(self.vertices)}\n")
          file.write("property float x\n")
          file.write("property float y\n")
          file.write("property float z\n")
          file.write(f"element face {len(self.faces)}\n")
          file.write("property list uchar uint vertex_indices\n")
          file.write("end_header\n")

          for vertex in self.vertices:
              file.write(f"{vertex.x} {vertex.y} {vertex.z}\n")

          for face in self.faces:
              formatted_vertices = ' '.join(str(v) for v in face.vertices)
              number_of_vertices = len(face.vertices)
              file.write(f"{number_of_vertices} {formatted_vertices}\n")


    def normalize(self):
        x_coordinates = [vertex.x for vertex in self.vertices]
        y_coordinates = [vertex.y for vertex in self.vertices]
        z_coordinates = [vertex.z for vertex in self.vertices]

        max_distance = max(x_coordinates + y_coordinates + z_coordinates)
        min_distance = min(x_coordinates + y_coordinates + z_coordinates)
        normalization_range = max_distance - min_distance

        if normalization_range == 0:
            raise ValueError("Normalization range cannot be zero")

        for vertex in self.vertices:
            vertex.x = 2 * (vertex.x - min_distance) / normalization_range - 1
            vertex.y = 2 * (vertex.y - min_distance) / normalization_range - 1
            vertex.z = 2 * (vertex.z - min_distance) / normalization_range - 1


    @classmethod
    def from_file(cls, filepath):
      with open(filepath, 'r') as f:
          if next(f).strip() != "ply":
              raise ValueError("The file being read is not a PLY file.")

          for _ in range(2):
              next(f)

          n_vertices = int(next(f).split()[-1])

          for _ in range(3):
              next(f)

          n_faces = int(next(f).split()[-1])

          for _ in range(2):
              next(f)

          vertices = []
          for _ in range(n_vertices):
              x, y, z = map(float, next(f).split())
              vertices.append(Vertex(x, y, z))

          faces = []
          for _ in range(n_faces):
              face_vertices = list(map(int, next(f).split()[1:]))
              faces.append(Face(face_vertices))

      name = os.path.splitext(os.path.basename(filepath))[0]

      return cls(name, vertices, faces)

In [39]:
#@title { vertical-output: true}
ply_objs = []
count = 0

for f in os.listdir(PLY):
    if f.endswith('.ply'):
        ply_obj = PlyObject.from_file(filepath=os.path.join(PLY, f))
        ply_objs.append(ply_obj)

        count += 1
        print(f'Processed as object: {ply_obj.name}')
        print(f'Vertice count: {len(ply_obj.vertices)}')
        print(f'Face count: {len(ply_obj.faces)}')
        print('-' * 50)

Processed as object: Coldheart
Vertice count: 16146
Face count: 27391
--------------------------------------------------
Processed as object: Vigilance Wing
Vertice count: 8472
Face count: 15549
--------------------------------------------------
Processed as object: RatKing
Vertice count: 12231
Face count: 24311
--------------------------------------------------
Processed as object: Riskrunner
Vertice count: 16819
Face count: 27633
--------------------------------------------------
Processed as object: Skyburner's oath
Vertice count: 3593
Face count: 5987
--------------------------------------------------
Processed as object: Prometheus Lens
Vertice count: 13902
Face count: 24150
--------------------------------------------------
Processed as object: Tractor Cannon
Vertice count: 54808
Face count: 35705
--------------------------------------------------
Processed as object: Wardcliff Coil
Vertice count: 16441
Face count: 30054
--------------------------------------------------
Processe

In [40]:
#@title { vertical-output: true}
for obj in ply_objs:
    min_val = float('inf')
    max_val = float('-inf')

    for v in obj.vertices:
        min_val = min(min_val, v.x, v.y, v.z)
        max_val = max(max_val, v.x, v.y, v.z)

    print(f"Object: {obj.name}")
    print(f"Overall minimum: {min_val}")
    print(f"Overall maximum: {max_val}")
    print('-' * 50)

Object: Coldheart
Overall minimum: -0.246695
Overall maximum: 0.664834
--------------------------------------------------
Object: Vigilance Wing
Overall minimum: -0.315861
Overall maximum: 0.679819
--------------------------------------------------
Object: RatKing
Overall minimum: -0.076467
Overall maximum: 0.31704
--------------------------------------------------
Object: Riskrunner
Overall minimum: -0.283287
Overall maximum: 0.456181
--------------------------------------------------
Object: Skyburner's oath
Overall minimum: -0.271624
Overall maximum: 0.418271
--------------------------------------------------
Object: Prometheus Lens
Overall minimum: -0.241697
Overall maximum: 0.686412
--------------------------------------------------
Object: Tractor Cannon
Overall minimum: -0.274935
Overall maximum: 0.31358
--------------------------------------------------
Object: Wardcliff Coil
Overall minimum: -0.289722
Overall maximum: 0.496558
--------------------------------------------------

In [42]:
#@title { vertical-output: true}

for obj in ply_objs:
    PlyObject.normalize(obj)

    min_val = float('inf')
    max_val = float('-inf')

    for v in obj.vertices:
        min_val = min(min_val, v.x, v.y, v.z)
        max_val = max(max_val, v.x, v.y, v.z)

    print(f"Object: {obj.name}")
    print(f"Overall minimum: {min_val}")
    print(f"Overall maximum: {max_val}")
    print('-' * 50)

Object: Coldheart
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Vigilance Wing
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: RatKing
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Riskrunner
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Skyburner's oath
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Prometheus Lens
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Tractor Cannon
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Wardcliff Coil
Overall minimum: -1.0
Overall maximum: 1.0
--------------------------------------------------
Object: Anarchy
Overall minimum: -1.0
Overall maximum: 1.0
------------------

In [43]:
!rm /content/drive/MyDrive/datasets/AutoCalibr/test.ply
ply_objs[4].save_file(DIR+"test.ply")

rm: cannot remove '/content/drive/MyDrive/datasets/AutoCalibr/test.ply': No such file or directory
